In [25]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from itertools import combinations
import DirtyFrame
import DropFrame

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

sys.path.append('../src')

import NoisyDataGeneration as ndg

[autoreload of DeepNeuralNetwork failed: Traceback (most recent call last):
  File "C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 309, in update_function
    setattr(old, name, getattr(new, name))
ValueError: f1_score() requires a code object with 0 free vars, not 2418066587651
]


In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
target_name = 'Hazardous'

"""Generazione Dataset per Decision Tree"""
df_tree = pd.read_csv('../input/nasaClean.csv')

features_tree = df_tree.columns.to_list()
features_tree.remove(target_name)

"""Generazione Dataset per Depp Neural Network"""
df_dnn = pd.read_csv('../input/nasaClean.csv')

features_dnn = df_dnn.columns.to_list()
features_dnn.remove(target_name)

columns_to_remove = ['Neo Reference ID', 'Name', 'Close Approach Date', 'Orbit Determination Date', 'Hazardous',
                     'Orbiting Body', 'Equinox']

for col in columns_to_remove:
    if col in features_dnn:
        features_dnn.remove(col)
        df_dnn = df_dnn.drop(col, axis=1)

In [28]:
target_name='Hazardous'
test_size=0.15

In [29]:
features_tree = df_tree.columns.tolist()
features_tree.remove(target_name)
features_dnn = df_dnn.columns.tolist()
features_dnn.remove(target_name)

In [30]:
dataframe = None

y = df_dnn[target_name]

X_tree_train_base, X_tree_test, y_tree_train, y_tree_test = \
    train_test_split(df_tree[features_tree], df_tree[target_name], test_size=test_size,
                     stratify=y,
                     random_state=42)

X_dnn_train_base, X_dnn_test, y_dnn_train, y_dnn_test = \
    train_test_split(df_dnn[features_dnn], df_dnn[target_name], test_size=test_size,
                     stratify=y,
                     random_state=42)

In [31]:
print(X_tree_train_base)

      Absolute Magnitude  Est Dia in KM(min)  Epoch Date Close Approach  \
1909                26.4            0.013949              1139385600000   
3855                27.7            0.007666              1394866800000   
2364                19.2            0.384198              1204963200000   
3463                17.9            0.699125              1350284400000   
2902                25.8            0.018389              1279782000000   
...                  ...                 ...                        ...   
3758                21.6            0.127220              1384502400000   
3954                23.6            0.050647              1405407600000   
2360                21.8            0.116026              1203667200000   
2547                22.4            0.088015              1231401600000   
301                 19.1            0.402305               853315200000   

      Relative Velocity km per sec  Miss Dist.(Astronomical)  Orbit ID  \
1909                     

In [32]:
datasets_tree = ndg.create_dirty_dataset(pd.DataFrame(X_tree_train_base, columns=features_tree))
datasets_dnn = ndg.create_dirty_dataset(pd.DataFrame(X_dnn_train_base, columns=features_dnn))

columns = ['Dataset Name', 'Tree Accuracy', 'Tree Precision', 'Tree Recall', 'Tree F1 Score',
           'Tree Clean Auc', 'Tree Clean Accuracy', 'Tree Clean Precision', 'Tree Clean Recall',
           'Tree Clean F1 Score', 'Tree Clean Auc',
           'DNN Accuracy', 'DNN Precision', 'DNN Recall', 'DNN F1 Score', 'DNN Auc']

In [13]:
for key, value in datasets_tree.items():
    keyNull = key
    break

In [14]:
error_type = keyNull[(key.find('-') + 1):keyNull.find('/')]
percentage = keyNull[(key.rfind('_') + 1):]

X_tree_train = datasets_tree[keyNull]
X_dnn_train = datasets_dnn[keyNull]

df_clean = pd.DataFrame(X_tree_train, columns=features_tree)
df_clean[target_name] = y_tree_train

df_clean = clean_data(df_clean)

X_tree_clean = df_clean[features_tree]

In [15]:
null_counts_per_row = X_dnn_train.isnull().sum()
print(null_counts_per_row)

Absolute Magnitude              199
Est Dia in KM(min)              199
Epoch Date Close Approach       199
Relative Velocity km per sec    199
Miss Dist.(Astronomical)        199
Orbit ID                        199
Orbit Uncertainity              199
Minimum Orbit Intersection      199
Jupiter Tisserand Invariant     199
Epoch Osculation                199
Eccentricity                    199
Semi Major Axis                 199
Inclination                     199
Asc Node Longitude              199
Orbital Period                  199
Perihelion Distance             199
Perihelion Arg                  199
Aphelion Dist                   199
Perihelion Time                 199
Mean Anomaly                    199
Mean Motion                     199
dtype: int64


In [33]:
def handle_missing_values(df, method='mean', value=None):
    """
    Gestisce i valori nulli nel dataframe.

    Parametri:
    df (DataFrame): Il dataframe da pulire.
    method (str): Il metodo per gestire i valori nulli ('drop', 'mean', 'median', 'mode', 'value').
    value (optional): Il valore con cui sostituire i valori nulli se method è 'value'.

    Ritorna:
    DataFrame: Il dataframe con i valori nulli gestiti.
    """
    if method == 'drop':
        df = df.dropna()
    elif method == 'mean':
        df = df.fillna(df.mean(numeric_only=True))
    elif method == 'median':
        df = df.fillna(df.median(numeric_only=True))
    elif method == 'mode':
        df = df.fillna(df.mode().iloc[0])
    elif method == 'value':
        df = df.fillna(value)
    return df


def handle_outliers(df, method='cap', factor=1.5):
    """
    Gestisce gli outliers nel dataframe.

    Parametri:
    df (DataFrame): Il dataframe da pulire.
    method (str): Il metodo per gestire gli outliers ('remove', 'cap').
    factor (float): Il fattore per il calcolo dell'IQR per il metodo 'cap'.

    Ritorna:
    DataFrame: Il dataframe con gli outliers gestiti.
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if method == 'remove':
        for col in numeric_cols:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - factor * iqr
            upper_bound = q3 + factor * iqr
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    elif method == 'cap':
        for col in numeric_cols:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - factor * iqr
            upper_bound = q3 + factor * iqr
            df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
            df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    return df


def handle_inconsistent_values(df, method='median', value=None):
    """
    Gestisce i valori inconsistenti nel dataframe.

    Parametri:
    df (DataFrame): Il dataframe da pulire.
    method (str): Il metodo per gestire i valori inconsistenti ('drop', 'mean', 'median', 'mode', 'value').
    value (optional): Il valore con cui sostituire i valori inconsistenti se method è 'value'.

    Ritorna:
    DataFrame: Il dataframe con i valori inconsistenti gestiti.
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if method == 'drop':
        for col in numeric_cols:
            df = df[np.isfinite(df[col])]
    elif method == 'mean':
        for col in numeric_cols:
            df[col] = np.where(~np.isfinite(df[col]), df[col].mean(), df[col])
    elif method == 'median':
        for col in numeric_cols:
            df[col] = np.where(~np.isfinite(df[col]), df[col].median(), df[col])
    elif method == 'mode':
        for col in numeric_cols:
            df[col] = np.where(~np.isfinite(df[col]), df[col].mode().iloc[0], df[col])
    elif method == 'value':
        for col in numeric_cols:
            df[col] = np.where(~np.isfinite(df[col]), value, df[col])
    return df


def clean_data(df, missing_method='mean', outlier_method='cap', inconsistent_method='median', value=None):
    """
    Pulisce i dati nel file CSV specificato.

    Parametri:
    df: Il dataframe da pulire
    missing_method (str): Il metodo per gestire i valori nulli.
    outlier_method (str): Il metodo per gestire gli outliers.
    inconsistent_method (str): Il metodo per gestire i valori inconsistenti.
    value (optional): Il valore con cui sostituire i valori nulli o inconsistenti se i metodi sono 'value'.

    Ritorna:
    DataFrame: Il dataframe pulito.
    """

    df_clean = df.copy()

    # Gestisce i valori nulli
    df_clean = handle_missing_values(df_clean, method=missing_method, value=value)

    # Gestisce gli outliers
    df_clean = handle_outliers(df_clean, method=outlier_method)

    # Gestisce i valori inconsistenti
    df_clean = handle_inconsistent_values(df_clean, method=inconsistent_method, value=value)



    return df_clean

In [17]:
X_tree_clean

,Absolute Magnitude,Est Dia in KM(min),Epoch Date Close Approach,Relative Velocity km per sec,Miss Dist.(Astronomical),Orbit ID,Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Epoch Osculation,...,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion
1909,26.4,0.013949,1.139386e+12,12.763999,0.140676,8.000000,7.0,0.003463,7.681,2458000.5,...,0.741179,9.231603,10.675944,233.068048,0.397750,335.461926,1.084607,2.457894e+06,165.092836,1.544613
3855,27.7,0.007666,1.394867e+12,16.702432,0.347988,28.534299,7.0,0.017187,4.334,2458000.5,...,1.561573,9.672530,145.892576,712.757927,0.979768,334.536391,2.143379,2.458125e+06,297.088720,0.754300
2364,19.2,0.384198,1.204963e+12,27.148424,0.492638,21.000000,1.0,0.223726,8.392,2458000.5,...,0.662809,20.370517,85.939420,197.097380,0.273430,248.151009,1.052188,2.457931e+06,127.222810,1.793178
3463,17.9,0.609464,1.350284e+12,17.611421,0.347922,57.000000,1.0,0.035299,3.441,2458000.5,...,2.003091,31.289721,44.682563,1035.498307,0.788094,291.194299,3.218088,2.458242e+06,276.148780,0.347659
2902,25.8,0.018389,1.279782e+12,9.860759,0.293526,7.000000,6.0,0.042449,6.642,2458000.5,...,0.891446,7.580554,127.694273,307.426026,0.719186,156.422464,1.063705,2.457848e+06,178.893467,1.171013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3758,21.6,0.127220,1.384502e+12,23.300262,0.191154,17.000000,1.0,0.104712,6.607,2458000.5,...,0.867466,31.720793,238.235731,295.105466,0.450272,31.332598,1.284660,2.458020e+06,188.150049,1.219903
3954,23.6,0.050647,1.405408e+12,12.062920,0.652421,9.000000,6.0,0.095708,6.065,2458000.5,...,0.992569,18.261338,98.255758,361.192997,0.861260,340.090321,1.123877,2.458060e+06,300.367883,0.996697
2360,21.8,0.209216,1.203667e+12,28.960201,0.436412,18.000000,4.0,0.213406,3.786,2458000.5,...,1.927967,14.846032,136.118262,977.794776,1.191130,17.601963,2.664804,2.458410e+06,209.075386,0.368175
2547,22.4,0.088015,1.231402e+12,14.427204,0.273358,16.000000,1.0,0.047490,5.967,2458000.5,...,0.986774,23.269477,284.777138,358.034697,0.495538,307.790347,1.478011,2.458145e+06,214.269486,1.005489


In [23]:
trained = DirtyFrame.DataFrame(error_type, percentage, X_tree_train, X_tree_test, y_tree_train, y_tree_test,
                               X_tree_clean, X_dnn_test, y_dnn_train, y_dnn_test, X_tree_clean, y_tree_train, df=df_dnn)

type:  <class 'pandas.core.series.Series'>
Epoch 1/10


[autoreload of DeepNeuralNetwork failed: Traceback (most recent call last):
  File "C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 309, in update_function
    setattr(old, name, getattr(new, name))
ValueError: f1_score() requires a code object with 0 free vars, not 2418066587651
]


125/125 [==============================] - 0s 594us/step - loss: 0.3180 - f1: 0.3616
Epoch 2/10
125/125 [==============================] - 0s 875us/step - loss: 0.2092 - f1: 0.5529
Epoch 3/10
125/125 [==============================] - 0s 563us/step - loss: 0.1801 - f1: 0.6405
Epoch 4/10
125/125 [==============================] - 0s 555us/step - loss: 0.1639 - f1: 0.6664
Epoch 5/10
125/125 [==============================] - 0s 542us/step - loss: 0.1517 - f1: 0.6939
Epoch 6/10
125/125 [==============================] - 0s 538us/step - loss: 0.1405 - f1: 0.7095
Epoch 7/10
125/125 [==============================] - 0s 538us/step - loss: 0.1277 - f1: 0.7290
Epoch 8/10
125/125 [==============================] - 0s 523us/step - loss: 0.1178 - f1: 0.7474
Epoch 9/10
125/125 [==============================] - 0s 567us/step - loss: 0.1090 - f1: 0.7650
Epoch 10/10
22/22 [==============================] - 0s 365us/step


In [38]:
    for key, value in datasets_tree.items():
        error_type = key[(key.find('-') + 1):key.find('/')]
        percentage = key[(key.rfind('_') + 1):]

        if error_type == 'null' or error_type == 'outlier':
            if percentage == 10 or percentage == '10':
                continue

        print(f'Start of {error_type}_{percentage} Dataset')

        X_tree_train = datasets_tree[key].values
        X_dnn_train = datasets_dnn[key].values

        df_clean = pd.DataFrame(X_tree_train, columns=features_tree)
        df_clean[target_name] = y_tree_train

        df_clean = clean_data(df_clean)

        X_tree_clean = df_clean[features_tree].values

        trained = DirtyFrame.DataFrame(error_type, percentage, X_tree_train, X_tree_test, y_tree_train, y_tree_test,
                                       X_tree_clean, X_dnn_test, y_dnn_train, y_dnn_test, X_tree_clean, y_tree_train,
                                       df=df_dnn)

        dir_name = key[0:key.find('/')]
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)

        trained.tree1.shap_init()
        trained.tree1.shap.print_beeswarm(dir_name + f'/{error_type}_{percentage}_tree')
        trained.tree2.shap_init()
        trained.tree2.shap.print_beeswarm(dir_name + f'/{error_type}_{percentage}_clean')
        trained.dnn.shap_init()
        trained.dnn.shap.print_beeswarm(dir_name + f'/{error_type}_{percentage}_dnn')

        txt_file_path = os.path.join(dir_name, f"{error_type}_{percentage}.txt")
        with open(txt_file_path, 'w') as file:
            file.write(f"{error_type} {percentage}%\n")
            file.write('Tree Metrics:\n')
            file.write(f'\tAccuracy: {trained.metrics_tree.accuracy}\n')
            file.write(f'\tPrecision: {trained.metrics_tree.precision}\n')
            file.write(f'\tRecall: {trained.metrics_tree.recall}\n')
            file.write(f'\tF1 Score: {trained.metrics_tree.f1_score}\n')
            file.write(f'\tAUC: {trained.metrics_tree.auc}\n')
            file.write('Tree Clean Metrics:\n')
            file.write(f'\tAccuracy: {trained.metrics_tree_clean.accuracy}\n')
            file.write(f'\tPrecision: {trained.metrics_tree_clean.precision}\n')
            file.write(f'\tRecall: {trained.metrics_tree_clean.recall}\n')
            file.write(f'\tF1 Score: {trained.metrics_tree_clean.f1_score}\n')
            file.write(f'\tAUC: {trained.metrics_tree_clean.auc}\n')
            file.write('Deep Neural Network Metrics:\n')
            file.write(f'\tAccuracy: {trained.metrics_dnn.accuracy}\n')
            file.write(f'\tPrecision: {trained.metrics_dnn.precision}\n')
            file.write(f'\tRecall: {trained.metrics_dnn.recall}\n')
            file.write(f'\tF1 Score: {trained.metrics_dnn.f1_score}\n')
            file.write(f'\tAUC: {trained.metrics_dnn.auc}\n')
        if dataframe is None:
            data = {
                'Dataset Name': [f"{error_type}{percentage}"],
                'Tree Accuracy': [trained.metrics_tree.accuracy],
                'Tree Precision': [trained.metrics_tree.precision],
                'Tree Recall': [trained.metrics_tree.recall],
                'Tree F1 Score': [trained.metrics_tree.f1_score],
                'Tree Auc': [trained.metrics_tree.auc],
                'Tree Clean Accuracy': [trained.metrics_tree_clean.accuracy],
                'Tree Clean Precision': [trained.metrics_tree_clean.precision],
                'Tree Clean Recall': [trained.metrics_tree_clean.recall],
                'Tree Clean F1 Score': [trained.metrics_tree_clean.f1_score],
                'Tree Clean Auc': [trained.metrics_tree_clean.auc],
                'DNN Accuracy': [trained.metrics_dnn.accuracy],
                'DNN Precision': [trained.metrics_dnn.precision],
                'DNN Recall': [trained.metrics_dnn.recall],
                'DNN F1 Score': [trained.metrics_dnn.f1_score],
                'DNN Auc': [trained.metrics_dnn.auc]
            }
            dataframe = pd.DataFrame(data=data, columns=columns)
        else:
            data = {
                'Dataset Name': f"{error_type}{percentage}",
                'Tree Accuracy': trained.metrics_tree.accuracy,
                'Tree Precision': trained.metrics_tree.precision,
                'Tree Recall': trained.metrics_tree.recall,
                'Tree F1 Score': trained.metrics_tree.f1_score,
                'Tree Auc': trained.metrics_tree.auc,
                'Tree Clean Accuracy': trained.metrics_tree_clean.accuracy,
                'Tree Clean Precision': trained.metrics_tree_clean.precision,
                'Tree Clean Recall': trained.metrics_tree_clean.recall,
                'Tree Clean F1 Score': trained.metrics_tree_clean.f1_score,
                'Tree Clean Auc': trained.metrics_tree_clean.auc,
                'DNN Accuracy': trained.metrics_dnn.accuracy,
                'DNN Precision': trained.metrics_dnn.precision,
                'DNN Recall': trained.metrics_dnn.recall,
                'DNN F1 Score': trained.metrics_dnn.f1_score,
                'DNN Auc': trained.metrics_dnn.auc
            }
            dataframe.loc[len(dataframe)] = data

        print(f'End of {error_type}_{percentage} Dataset')

    dataframe.to_csv('../TrainingSetsDir/dirtysets_trained_metrics.csv', index=False)

Start of inconsistent_10 Dataset


C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


type:  <class 'pandas.core.series.Series'>
1/1 [==============================] - 0s 24ms/step


  0%|          | 0/250 [00:00<?, ?it/s]

1305/1305 [==============================] - 1s 396us/step
End of inconsistent_10 Dataset
Start of all_10 Dataset


...

1307/1307 [==============================] - 1s 404us/step
End of oi_90 Dataset
Start of in_90 Dataset


C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\Garnet14\PycharmProjects\DataManagementAsteroids\venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


type:  <class 'pandas.core.series.Series'>
1/1 [==============================] - 0s 26ms/step


  0%|          | 0/250 [00:00<?, ?it/s]

1307/1307 [==============================] - 1s 395us/step
End of in_90 Dataset
